In [1]:
import urllib.request
from bs4 import BeautifulSoup as soup
import requests
import re
import pandas as pd
import dataframe_image as dfi


link = 'https://rally-base.com/2023/rally-italia-sardegna-2023/?ssId='
startat, no_ss=7596, int(19) # starting number of url, count of Special Stages
canceled = []

stages = [number for number in range(no_ss)]


rally_23 = []

In [2]:
if canceled:
    for j in canceled: stages.remove(j-1)
#print(stages)


In [3]:
for ss in stages:
    val= startat + ss
    ss_a = str(val)
    my_url11 = link + ss_a
    #print(ss, val, ss_a, "\n", my_url11)
    
    req = urllib.request.Request(my_url11, headers={'User-Agent': 'Mozilla/5.0'})
    uClient11 = urllib.request.urlopen(req)
    page_html11 = uClient11.read()
    uClient11.close()
    data = pd.read_html(page_html11)[1]
   
    data.columns = data.iloc[0]
    data = data[1:]
    data['ss']=ss+1
    #print(data.columns)
    
    equal = '-' in data['Pos.'].unique()
    if equal:
        data['Pos.'] = data['Pos.'].replace('-', method='ffill')
    
    #data.to_csv('05_rally23_SS'+str(ss+1)+'.csv', index=False)

    rally_23.append(data)

In [4]:
rally2023_stages = pd.concat(rally_23, axis=0)
#rally2023_stages['Pos.'] = rally2023_stages['Pos.'].astype(str).astype(int)
rally2023_stages['No.'] = rally2023_stages['No.'].astype(str).astype(int)
#rally2023_stages.to_csv('03_rally2023.csv', index=False)
rally2023_stages = rally2023_stages.fillna("-")
rally2023_temp=rally2023_stages.drop(['Group','SS time', 'Aver. speed sec/km', 'Diff.Leader Diff.Prev.'], axis=1)
rally2023_view = rally2023_temp.set_index(['No.','Driver / Co-driver Vehicle','ss'], drop=True).unstack('ss')

rally2023_temp2 = rally2023_temp[(rally2023_temp['No.']<30)]
rally2023_temp3 = rally2023_temp[(rally2023_temp['No.']==69)]
#rally2023_temp2.append(rally2023_temp3)
#rally2023_temp4 = rally2023_temp2.append(rally2023_temp3)
rally2023_temp4 = pd.concat([rally2023_temp2, rally2023_temp3],axis=0)
rally2023_view2 = rally2023_temp4.set_index(['No.','Driver / Co-driver Vehicle','ss'], drop=True).unstack('ss')
rally2023_view2 = rally2023_view2.fillna("-")
rally2023_view2

0                                                      Pos.                  \
ss                                                       1   2   3   4   5    
No. Driver / Co-driver Vehicle                                                
4   Lappi  Esapekka  /  Ferm  Janne  Hyundai i20 N ...    1   2   5   3   1   
6   Sordo  Dani  /  Carrera  C.  Hyundai i20 N Rall...    7   7   2  23   8   
7   Loubet  P.  /  Gilsoul  Nicolas  Ford Puma Rall...    9   3   4   4   2   
8   Tänak  Ott  /  Järveoja  M.  Ford Puma Rally1 H...    2   8   6   7   6   
11  Neuville  T.  /  Wydaeghe  M.  Hyundai i20 N Ra...    3   5   3   5   3   
17  Ogier  S.  /  Landais  Vincent  Toyota GR Yaris...    6   1   7   1   4   
18  Katsuta  T.  /  Johnston  Aaron  Toyota GR Yari...    4   4   1   6   9   
20  Rossel  Yohan  /  Dunand  Arnaud  Citroën C3 Ra...   22  19  24  14  10   
21  Greensmith  Gus  /  Andersson  Jonas  Škoda Fab...   25  13  14  15  10   
22  Solberg  Oliver  /  Edmondson  E.  Škoda Fabia ...   10  10  14  68  10   
23  Mikkelsen  A.  /  Eriksen  T.  Škoda Fabia RS R...   12  11  11  12  10   
24  Gryazin  Nikolay  /  Aleksandrov  K.  Škoda Fab...   23  35  18  32  10   
25  Suninen  Teemu  /  Markkula  Mikko  Hyundai i20...   29  12  10   9  10   
26  Lindholm  Emil  /  Hämäläinen  R.  Škoda Fabia ...   14  17  12  13  10   
27  Fourmaux  Adrien  /  Coria  Alexandre  Ford Fie...   19  14  13  10  10   
28  Ciamin  Nicolas  /  Roche  Yannick  Volkswagen ...   31  29  30  51  58   
29  Kajetanowicz  K.  /  Szczepaniak  M.  Škoda Fab...   13  20  25  16  10   
69  Rovanperä  K.  /  Halttunen  Jonne  Toyota GR Y...    5   6   8   8   7   

0                                                                           \
ss                                                      6   7   8   9   10   
No. Driver / Co-driver Vehicle                                               
4   Lappi  Esapekka  /  Ferm  Janne  Hyundai i20 N ...   2   2   4   2   3   
6   Sordo  Dani  /  Carrera  C.  Hyundai i20 N Rall...   7   5   8   6   2   
7   Loubet  P.  /  Gilsoul  Nicolas  Ford Puma Rall...   6  38   -   -   -   
8   Tänak  Ott  /  Järveoja  M.  Ford Puma Rally1 H...   4   7   6  11  56   
11  Neuville  T.  /  Wydaeghe  M.  Hyundai i20 N Ra...   1   3   1   1   5   
17  Ogier  S.  /  Landais  Vincent  Toyota GR Yaris...   5   4   2   3   1   
18  Katsuta  T.  /  Johnston  Aaron  Toyota GR Yari...   3   6  11  65  56   
20  Rossel  Yohan  /  Dunand  Arnaud  Citroën C3 Ra...  12  17  22  22   9   
21  Greensmith  Gus  /  Andersson  Jonas  Škoda Fab...  57  43   -   -   -   
22  Solberg  Oliver  /  Edmondson  E.  Škoda Fabia ...  12   9   7  10   7   
23  Mikkelsen  A.  /  Eriksen  T.  Škoda Fabia RS R...  12  13   9   7  11   
24  Gryazin  Nikolay  /  Aleksandrov  K.  Škoda Fab...  12  14  16  13  14   
25  Suninen  Teemu  /  Markkula  Mikko  Hyundai i20...  11  24  10   9  12   
26  Lindholm  Emil  /  Hämäläinen  R.  Škoda Fabia ...  12  11  14  19  30   
27  Fourmaux  Adrien  /  Coria  Alexandre  Ford Fie...  10  12  12   8   6   
28  Ciamin  Nicolas  /  Roche  Yannick  Volkswagen ...  57  43  25  26  19   
29  Kajetanowicz  K.  /  Szczepaniak  M.  Škoda Fab...  12  21  18  12   8   
69  Rovanperä  K.  /  Halttunen  Jonne  Toyota GR Y...   8   1   3   4   4   

0                                                                           \
ss                                                      11  12  13  14  15   
No. Driver / Co-driver Vehicle                                               
4   Lappi  Esapekka  /  Ferm  Janne  Hyundai i20 N ...   5   1   3   3   4   
6   Sordo  Dani  /  Carrera  C.  Hyundai i20 N Rall...   4   6   5   1   2   
7   Loubet  P.  /  Gilsoul  Nicolas  Ford Puma Rall...   -   -   -   -   -   
8   Tänak  Ott  /  Järveoja  M.  Ford Puma Rally1 H...  55  52  48  44  44   
11  Neuville  T.  /  Wydaeghe  M.  Hyundai i20 N Ra...   1   2   1   2   1   
17  Ogier  S.  /  Landais  Vincent  Toyota GR Yaris...   2   3   4  44  44   

In [5]:
pd.set_option("display.max_columns", None)
rally2023_view2
dfi.export(rally2023_view2, 'Sardegna_Stages_Classification.jpg')